In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
BASE_PATH = os.path.join('drive', 'My Drive', '모비컴 2021')
DATA_PATH = os.path.join(BASE_PATH, 'cafedata')
# NOTEBOOKS_PATH = os.path.join(BASE_PATH, 'Notes')
print(os.listdir(BASE_PATH))
print(os.listdir(DATA_PATH))

['인터뷰 질문지 supplementary material.gdoc', '취약지역.pdf', '인터뷰 질문지.gdoc', '3 17.gdoc', '3 19.gdoc', 'Team Exercise (03 11).gslides', '3 25.gdoc', '3 31 - 이영기 교수님 미팅.gdoc', '210319_미팅.gdoc', '4 8_문진 로직 문의.gdoc', '산전관리 데이터 수집 관리 어플리케이션 개요.gdoc', '의사선생님들이 들려주시는 각종 산부인과 의학상식.gdoc', '4 16.gdoc', '외래 간호사 인터뷰.gdoc', '4 23.gdoc', '210427 Paper Review_Student Life_Team4.pptx', '210427 Paper Review_Student Life_Team4.pdf', '외래 진찰 참관 속기록.gsheet', 'Proposal Draft.gslides', 'Project Schedule.gsheet', 'Midterm Presentation Draft.gslides', 'mobicom figure.pptx', '챗봇 알러뷰봇 동작 안내.gdoc', '5 14.gdoc', 'Lecture   Paper Presentation List.gsheet', '사용 시나리오.gdoc', 'cafe.gsheet', '카페 분석.gsheet', 'cafedata', 'Analysis.ipynb']
['P5_55566357.json', 'P5_55558112.json', 'P5_55543623.json', 'P5_55539035.json', 'P5_55511452.json', 'P5_55502459.json', 'P5_5550108

In [ ]:
filenames = [i for i in os.listdir(DATA_PATH) if i.endswith("json")]
len(filenames)

1233

In [ ]:
# import json
# data = []
# for filename in filenames:
#   filepath = os.path.join(DATA_PATH, filename)
#   with open(filepath) as f:
#     data.append(json.loaTd(f))
# len(data)

1233

In [ ]:
# save_path = os.path.join(BASE_PATH, "cafedata_unified.json")
# with open(save_path, "w") as f:
#   f.write(json.dumps(data))

In [ ]:
data = None
with open(save_path) as f:
  data = json.load(f)
data[0]

{'author': '줌마양이',
 'contents': ['생후 25일이구요,',
  '아이엠마더 조리원서 부터 먹다가 집에오니 배앓이심하고 녹변보길래 압타밀로 바꾼다고 그제부터 시도했어요. 그제 한번 어제 두번 먹이고  변은 황금색으로 잘보던데 오늘 아침에 한번먹고 소화가 잘되서 허기진지 한시간 좀 지나니 배고파하더라구요. 그래서 아이엠마더 먹였는데 또 한시간 반 지나서 달라는 것 같아서 아이엠 마더 먹이는도중에 분수토를 쫙! 이래도 괜찮은 건지 모르겠네요. 이것저것 먹어서 애 소화력에 문제생긴건지 ㅜㅜ 분수토는 첨봐서 당황스러워요.'],
 'date': '2021.05.07. 11:42',
 'id': '55566357',
 'title': '분유갈아탄다고 퐁당퐁당 먹이다가',
 'title_category': '[0ㅡ1개월]'}

In [ ]:
sentences = sum([[entry['title'], *entry['contents']] for entry in data], [])
len(sentences), sentences[0]

(8321, '분유갈아탄다고 퐁당퐁당 먹이다가')

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 460kB 24.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import logging

from konlpy.tag import Okt

okt = None


def load_tagger():
    global okt
    logging.info('Initializing Okt pos-tagger')
    okt = Okt()
    okt.pos('엔진 초기화용 문장')


class MorphTokenizer:
    def __init__(self):
        self._tagger = None
        self.stoplist = ['것', '적', '한다', '예컨대', '예']
        self.excluded_tags = ['Josa', 'Punctuation', 'Foreign']

    @property
    def tagger(self):
        if self._tagger is None:
            if okt is None:
                load_tagger()
            self._tagger = okt
        return self._tagger

    def tokenize(self, document):
        return [
            f'{word}/{tag}' for word, tag in self.tagger.pos(document)
            if tag not in self.excluded_tags and word not in self.stoplist
        ]

In [ ]:
import logging

from gensim import corpora, models, similarities
from gensim.matutils import corpus2dense

class Vectorizer:
    def train(self, corpus):
        pass

    def get_sentence_vector(self, sentence):
        pass

    def get_sentence_vectors(self, sentences):
        return [
            self.get_sentence_vector(sentence)
            for sentence in sentences
        ]

class LsiVectorizer(Vectorizer):
    def __init__(self):
        logging.info('Start initializing LsiVectorizer...')
        self.dictionary = None
        self.tokenizer = MorphTokenizer()

    def train(self, corpus):
        tokenized_corpus = [
            self.tokenizer.tokenize(document)
            for document in corpus
        ]
        # build dictionary
        self.dictionary = corpora.Dictionary(tokenized_corpus)
        # remove words that appear only once
        self.dictionary.filter_tokens([
            token_id for token_id, doc_freq in self.dictionary.dfs.items()
            if doc_freq == 1
        ])
        # build corpus
        self.corpus = [
            self.dictionary.doc2bow(tokenized_document)
            for tokenized_document in tokenized_corpus
        ]
        self.model_tfidf = models.TfidfModel(self.corpus)
        self.corpus_tfidf = self.model_tfidf[self.corpus]
        self.model_lsi = models.LsiModel(
            self.corpus_tfidf, id2word=self.dictionary, num_topics=20
        )
        self.num_topics, num_features = self.model_lsi.get_topics().shape
        logging.info(f'LSI num_topics: {self.num_topics}')
        # └> min(num_topics=250, num_documents)
        self.corpus_lsi = self.model_lsi[self.corpus_tfidf]
        self.index_lsi = similarities.MatrixSimilarity(
            self.corpus_lsi, num_features=num_features)
        # └> self.index_lsi[lsi_vector] => similarities to each document

    def get_sentence_vector(self, sentence):
        tokenized_sentence = self.tokenizer.tokenize(sentence)
        # └> ['못/Noun', '찾겠다/Verb', '이번/Noun', ... ]
        bow_sentence = self.dictionary.doc2bow(tokenized_sentence)
        # └> [(112, 1), (178, 1), (284, 1), (471, 1), ... ]
        tfidf_vector = self.model_tfidf[bow_sentence]
        # └> : [(112, 0.19214051340327296), (178, 0.23739237055277979), ... ]
        lsi_vector = self.model_lsi[tfidf_vector]
        # └> : [(0, 0.021546872908136172), (1, 0.004916200019640246), ... ]
        # └> : length = self.num_topics
        dense_vector = corpus2dense([lsi_vector], self.num_topics)[:, 0]
        return dense_vector

In [ ]:
vectorizer = LsiVectorizer()
vectorizer.train(sentences)

In [ ]:
vectorizer.model_lsi.print_topics(-1)

[(0,
  '0.402*"ㅠㅠ/KoreanParticle" + 0.274*"주/Noun" + 0.249*"요/Noun" + 0.171*"배/Noun" + 0.160*"오늘/Noun" + 0.158*"아기/Noun" + 0.153*"너무/Adverb" + 0.144*"안/VerbPrefix" + 0.125*"임신/Noun" + 0.120*"다/Adverb"'),
 (1,
  '-0.589*"ㅠㅠ/KoreanParticle" + 0.572*"주/Noun" + 0.185*"오늘/Noun" + 0.159*"3일/Number" + 0.130*"2/Number" + 0.130*"차/Noun" + -0.108*"너무/Adverb" + 0.098*"검사/Noun" + 0.097*"1/Number" + 0.092*"16/Number"'),
 (2,
  '-0.549*"ㅠㅠ/KoreanParticle" + 0.490*"요/Noun" + -0.327*"주/Noun" + 0.162*"될까/Verb" + 0.123*"아기/Noun" + 0.112*"안/VerbPrefix" + 0.108*"할까/Verb" + -0.102*"배/Noun" + 0.101*"저/Noun" + -0.098*"오늘/Noun"'),
 (3,
  '-0.712*"배/Noun" + 0.330*"ㅠㅠ/KoreanParticle" + 0.295*"요/Noun" + -0.169*"너무/Adverb" + -0.144*"느낌/Noun" + -0.139*"아랫/Modifier" + -0.128*"통증/Noun" + 0.125*"주/Noun" + -0.110*"좀/Noun" + -0.105*"뭉침/Verb"'),
 (4,
  '0.540*"요/Noun" + 0.386*"배/Noun" + -0.310*"아기/Noun" + -0.189*"너무/Adverb" + -0.186*"다/Adverb" + 0.180*"될까/Verb" + -0.162*"들/Verb" + 0.155*"주/Noun" + -0.148*"들/Suffix" + -0

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(sentences)
# print(vectorizer.get_feature_names())

['05', '0세반', '0에서', '0이', '0이되어서', '0이라는뜻이', '0일', '0일이', '0일이구요', '10', '100', '1000가지', '1000미리', '1000배', '1000원을', '100g', '100g은', '100g임당재검', '100그람', '100도에서', '100먹고', '100미리', '100미리라고', '100미리정도', '100원', '100원을', '100은', '100이', '100이라고', '100이상', '100일', '100정도까지', '100찍고', '100평', '103일', '104', '105', '10kg까지', '10개월', '10개월동안', '10년이상', '10단위인걱', '10미리는', '10번', '10번하다', '10분', '10분간격으로', '10분기다려서', '10분마다', '10분만', '10분소요라던데', '10분쉬고', '10분여', '10시', '10시경', '10시에', '10시쯤', '10시쯤이', '10월', '10월이고', '10일', '10일만에', '10일밖에', '10일분', '10일정도까지', '10주', '10주까지', '10주되고서는', '10주되니', '10주에요', '10주인데', '10주인데요', '10주쯤부터', '10주차때', '10키로만', '10회담주에', '11', '110', '110까지', '110미리', '112', '112에', '11st', '11개월', '11번가에서', '11시', '11시30분에', '11시에', '11에', '11월3일에', '11월5일부터', '11월말이나', '11월은', '11일에', '11일째', '11일째인데', '11일차에요', '11주', '11주까지는', '11주인데', '11주차', '11주차인데', '11키로', '12', '120', '120미리', '120씩', '120프로가', '125를', '125일', '127', '12가트', '12시', '12시간만에', '12시반', '12시반

In [ ]:
tokenized_sentences = [
    vectorizer.tokenizer.tokenize(document)
    for document in sentences
]

In [ ]:
tokenized_sentences[0]

['분유/Noun', '갈아탄다고/Verb', '퐁당퐁당/Adverb', '먹이다가/Verb']

In [ ]:
from collections import Counter
cnt = Counter(sum(tokenized_sentences, []))

In [ ]:
l = []
for tid, token in vectorizer.dictionary.id2token.items():
  tflike = cnt[token]
  idf = vectorizer.model_tfidf.idfs[tid]
  l.append((token, tflike / idf))

l.sort(key=lambda x:-x[1])
l[:200]

[('ㅠㅠ/KoreanParticle', 141.28820422026342),
 ('요/Noun', 98.9166155099829),
 ('주/Noun', 94.36705918894089),
 ('아기/Noun', 63.34715336436409),
 ('안/VerbPrefix', 62.33065748315698),
 ('오늘/Noun', 55.04055934292189),
 ('배/Noun', 49.8220841858372),
 ('때/Noun', 48.592407601612784),
 ('다/Adverb', 46.054971404903355),
 ('너무/Adverb', 45.65740182869165),
 ('병원/Noun', 41.63294832871842),
 ('임신/Noun', 40.104641808976204),
 ('들/Suffix', 38.88536483790267),
 ('정도/Noun', 34.55186707422298),
 ('제/Noun', 34.02379645662998),
 ('좀/Noun', 32.29438958298602),
 ('출산/Noun', 31.5975889304001),
 ('잘/Verb', 29.692753832606165),
 ('이/Determiner', 28.152953993975476),
 ('지금/Noun', 27.321373564428672),
 ('ㅠ/KoreanParticle', 26.52913909185746),
 ('더/Noun', 26.18682761970446),
 ('집/Noun', 24.84700618372516),
 ('2/Number', 24.265015289392625),
 ('하는데/Verb', 24.13738653220864),
 ('한/Determiner', 23.799440110953185),
 ('전/Noun', 23.42007830682976),
 ('저/Noun', 23.41862754973713),
 ('후/Noun', 23.336039591471604),
 ('ㅜㅜ/Ko